In [19]:
import time
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import array
import numpy as np
from nvidia.dali.pipeline import Pipeline

class nvJPEGPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(nvJPEGPipeline, self).__init__(batch_size, num_threads, device_id, seed = 12)
        self.input = ops.FileReader(file_root = "/nvme/bm/img", file_list="/nvme/bm/list")
        self.decode = ops.nvJPEGDecoder(device = "mixed", output_type = types.GRAY,
                                        use_batched_decode=True)
        self.resize = ops.Resize(device="gpu",image_type = types.GRAY, resize_x=45., resize_y=55.)
        self.cast = ops.Cast(device="gpu", dtype=types.FLOAT)
        
    def define_graph(self):
        jpegs, labels = self.input(name="Reader")
        images = self.decode(jpegs)
        images = self.resize(images) 
        images = self.cast(images)
        # images are on the GPU
        return (images, labels)

In [20]:
BATCH_SIZE=8192
pipe = nvJPEGPipeline(BATCH_SIZE, 8, 0)
pipe.build()
print pipe.epoch_size()

#pipe_out = pipe.run()
#images, labels = pipe_out
#b=images.asCPU()
#x=b
#print x.at(0)
#x.at(0).shape

{u'Reader': 202599L}


In [21]:
#print images
print pipe.epoch_size("Reader") 
iters = pipe.epoch_size("Reader")
iters_tmp = iters
iters = iters // BATCH_SIZE
if iters_tmp != iters * BATCH_SIZE:
    iters += 1
    iters_tmp = iters

print iters

for j in range(iters):
            pipe._start_run()
            images, labels = pipe.outputs()
            #images, labels = pipe.run()
            print j
            print time.ctime(time.time())
            ##a=images.as_tensor()
            ##print a.shape()
            ##print a.dtype()
            a=images.asCPU()
            c=a.as_array()
            
            with open("/nvme/bm/celebA_rev3.bin_notebook_one", "ab") as f:
                #f.write(array.array("f", c.ravel()))
                np.asarray(c, dtype=np.float32).tofile(f)
f.close()

            
            


202599
25
0
Tue Feb 12 20:46:02 2019
1
Tue Feb 12 20:46:03 2019
2
Tue Feb 12 20:46:03 2019
3
Tue Feb 12 20:46:04 2019
4
Tue Feb 12 20:46:04 2019
5
Tue Feb 12 20:46:05 2019
6
Tue Feb 12 20:46:05 2019
7
Tue Feb 12 20:46:06 2019
8
Tue Feb 12 20:46:06 2019
9
Tue Feb 12 20:46:07 2019
10
Tue Feb 12 20:46:07 2019
11
Tue Feb 12 20:46:08 2019
12
Tue Feb 12 20:46:08 2019
13
Tue Feb 12 20:46:09 2019
14
Tue Feb 12 20:46:09 2019
15
Tue Feb 12 20:46:10 2019
16
Tue Feb 12 20:46:10 2019
17
Tue Feb 12 20:46:11 2019
18
Tue Feb 12 20:46:11 2019
19
Tue Feb 12 20:46:12 2019
20
Tue Feb 12 20:46:12 2019
21
Tue Feb 12 20:46:13 2019
22
Tue Feb 12 20:46:13 2019
23
Tue Feb 12 20:46:14 2019
24
Tue Feb 12 20:46:14 2019


In [ ]:
images, labels = pipe_out
print("Images is_dense_tensor: " + str(images.is_dense_tensor()))
print("Labels is_dense_tensor: " + str(labels.is_dense_tensor()))


In [ ]:
import array
a=images.asCPU()
c=a.as_array()
with open("/nvme/bm/celebA_rev3.bin_notebook", "wb") as f:
   f.write(array.array("f", c.ravel()))

In [ ]:
import numpy as np

labels_tensor = labels.as_tensor()

print (labels_tensor.shape())
#print (np.array(labels_tensor))

In [ ]:
#print images.asCPU()
#print (images_tensor.shape())
#print (np.array(images_tensor).shape)
#a= (np.array(images_tensor)[1]).reshape(218,178)
#print(images_tensor.asCPU())
#plt.imshow(a,cmap='gray')
#plt.show()

In [ ]:
pipe_out = pipe.run()
images, labels = pipe_out
a=images.asCPU()

In [ ]:
from __future__ import division
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def show_images(image_batch):
    columns = 4
    #rows = (batch_size + 1) // (columns)
    rows=4
    fig = plt.figure(figsize = (32,(32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns):
        plt.subplot(gs[j])
        plt.axis("off")
        #plt.imshow(image_batch.at(j),cmap="gray")
        #a=(np.array(b)[j])
        #a=a.reshape(218,178)
        plt.imshow( np.array(image_batch.at(j).reshape(55,45) ) ,cmap='gray',norm=None  )

In [ ]:
a.at(0)[0][0]=0.0
#a.at(0)[0][0]
show_images(a)


In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from PIL import Image

#fname = '/home/bm/1.jpg'
#image = Image.open(fname).convert("L")
#print image
#arr = np.asarray(image)
#print arr
#plt.imshow(arr, cmap='gray')
#plt.show()